# Notebook de préparation au modèle d'estimation

## Imports et paramètrage

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
from classes.credentials import Credentials as cr
from classes.prediction import ImmothepPrediction
from classes.split import Splitter
from sklearn.ensemble import IsolationForest

splitter = Splitter()
predictor = ImmothepPrediction()

In [2]:
valeurs2020 = pd.read_csv(cr.PATH+'2020.txt', sep='|', usecols=['Nature mutation', 'Code postal','Nombre pieces principales', 'Surface terrain', 'Valeur fonciere', 'Type local', 'Nombre de lots', 'Surface reelle bati'], encoding='utf-8')

## Nettoyage du dataset de biens immobiliers 2020
- Retirer les nuls
- Ploter
- Valeurs abhérentes
- Isolation forest

In [3]:
# Drop les lignes avec des nulles : type local, nombre lots, nb pièces, nature mutation, valeur foncière et CP
valeurs2020 = valeurs2020.dropna(subset = ['Type local', 'Nombre de lots', 'Nombre pieces principales', "Nature mutation", 'Valeur fonciere', 'Code postal'])

# On remplace les null de la surface de terrain par 0
valeurs2020['Surface terrain'] = valeurs2020['Surface terrain'].fillna(0) 

valeurs2020.head()

,Nature mutation,Valeur fonciere,Code postal,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
11,Vente,"72000,00",1270.0,0,Maison,35.0,2.0,381.0
13,Vente,"180300,00",1000.0,0,Maison,75.0,4.0,525.0
14,Vente,"54800,00",1000.0,2,Appartement,32.0,1.0,0.0
16,Vente,"350750,00",1000.0,0,Maison,201.0,7.0,1267.0
17,Vente,"350750,00",1000.0,0,Maison,201.0,7.0,1497.0


In [4]:
# Premier tri, on garde les appartements et maisons, les ventes, un nombre de pièces principales supérieur à 0, 
# un nombre de lots inférieur à 6, surface réelle bâtie supérieure à 8 (minimum légal)
valeurs2020 = valeurs2020[[
    'Type local',
    'Code postal', 
    'Nombre pieces principales', 
    'Surface terrain', 
    'Surface reelle bati',
    'Valeur fonciere'
    ]][valeurs2020['Type local'].str.contains("Appartement|Maison", regex=True)][valeurs2020['Nature mutation'].str.contains("Vente", regex=True)][valeurs2020['Nombre de lots']<6][valeurs2020['Nombre pieces principales']>0][valeurs2020['Surface reelle bati']>=8]


C:\Users\audre\AppData\Local\Temp/ipykernel_19576/2911758537.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valeurs2020 = valeurs2020[[


In [5]:
valeurs2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915817 entries, 11 to 2459557
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Type local                 915817 non-null  object 
 1   Code postal                915817 non-null  float64
 2   Nombre pieces principales  915817 non-null  float64
 3   Surface terrain            915817 non-null  float64
 4   Surface reelle bati        915817 non-null  float64
 5   Valeur fonciere            915817 non-null  object 
dtypes: float64(4), object(2)
memory usage: 48.9+ MB


In [6]:
# Changement des virgules en points
valeurs2020[['Valeur fonciere']] = valeurs2020[['Valeur fonciere']].replace(',', '.', regex=True)

# Passage des objets en floats
valeurs2020[['Valeur fonciere']]  = valeurs2020[['Valeur fonciere']].astype('float')

valeurs2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915817 entries, 11 to 2459557
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Type local                 915817 non-null  object 
 1   Code postal                915817 non-null  float64
 2   Nombre pieces principales  915817 non-null  float64
 3   Surface terrain            915817 non-null  float64
 4   Surface reelle bati        915817 non-null  float64
 5   Valeur fonciere            915817 non-null  float64
dtypes: float64(5), object(1)
memory usage: 48.9+ MB


In [7]:
# Séparation des maisons et appartements
valeursMaisons = valeurs2020[valeurs2020['Type local'].str.contains("Maison", regex=True)]
valeursApparts = valeurs2020[valeurs2020['Type local'].str.contains("Appartement", regex=True)]

# Import en CSV dans src/data/currated
valeursMaisons.to_csv(cr.CURATED_LOCAL_PATH + 'filtreMaisons2020.csv', index=False)
valeursApparts.to_csv(cr.CURATED_LOCAL_PATH + 'filtreAppart2020.csv', index=False)

## Split du dataset en départements

In [9]:
# Split des données par type de logement et par département
splitter.split_datas('filtreMaisons2020.csv', 'Code postal', 'CPMaisons')
splitter.split_datas('filtreAppart2020.csv', 'Code postal', 'CPAppart')

## Etalonage d'un modèle de régression linéaire

## Test du modèle